#Preprocesamiento de los datos

El siguiente documento incluye los pasos llevados a cabo, para realizar el preprocesamiento del texto de las noticias a utilizar en el trabajo "Evaluación de la precisión de los modelos de
clasificación binaria para la identificación de
noticias verdaderas o falsas en Costa Rica". Como archivo de entrada se requiere del uso del archivo "RawData_Espannol_Ingles_09222019.csv"

# **Librerias utilizadas**

In [ ]:
from nltk.corpus import stopwords

In [ ]:
#Import necessary packages, torch, numpy, pylab
import torch
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.init as init

#unicode to ASCII conversion
from __future__ import unicode_literals, print_function, division

#Import necessary packages, torch, numpy, pylab
import torch
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.init as init
import random
import torch.nn as nn
import matplotlib.ticker as ticker
#open files
from io import open
#global path
import glob
#Operating system calls
import os
#unicode manipulation
import unicodedata
#string manipulation
import string
import math

!pip3 install matplotlib2tikz
import matplotlib2tikz
import pandas as pd;
from numpy.random import choice
from sklearn.decomposition import PCA
from sklearn import preprocessing
import re
from unicodedata import normalize
from nltk.stem import LancasterStemmer, WordNetLemmatizer


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/Maestria/Proyecto de Investigación II/Dataset_files/webhose_dataset/"
rootDir = "/content/drive/My Drive/Maestria/Proyecto de Investigación II/Dataset_files/webhose_dataset/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 ConjuntoDatosPreprocesados_20190922.csv
'~$ConjuntoDatosPreprocesados_20190922_lematizado.xlsx'
 ConjuntoDatosPreprocesados_20190922_lematizado.xlsx
 ConjuntoDatosPreprocesados_20190922_v2.csv
 ConjuntoDatosPreprocesados_20190922_v2.xlsx
 ConjuntoDatosPreprocesados_20190922.xlsx
 ConjuntoDatosVersion1.csv
 ConjuntoDatosVersion1.xlsx
 ConjuntoDatosVersion2.csv
 ConjuntoDatosVersion2.xlsx
 ConjuntoDatosVersion3.csv
 ConjuntoDatosVersion3.xlsx
 ConjuntoDatosVersionWebScrapping_v1.csv
 ConjuntoDatosWebScrapping_v1.xlsx
 ConjuntoDatosWebScrapping_v2.xlsx
 ConjuntoFiltroRemoviendoFeatures.csv
'Copia de RawData_Espannol_Ingles_09222019_formulas.xlsx'
'datasource_clasificado_webhouse - copia.xlsx'
'datasource_clasificado_webhouse _Ingles.xlsx'
 datasource_clasificado_webhouse.xls
 datasource_clasificado.xls
'~$DescripcionVariables.xlsx'
 DescripcionVariables.xlsx
 D

In [ ]:
inputDataset = rootDir + 'RawData_Espannol_Ingles_09222019.csv';
df = pd.read_csv(inputDataset, sep=';', encoding='latin-1',header=(0),decimal=',' )
df.head()

,URLNoticia,Fuente,Clasificacion,MedioPeriodistico,Titulo,Texto,TipoSitio,TituloIngles,TextoIngles
0,http://columbia.co.cr/index.php/reportajes/esp...,http://columbia.co.cr/,1.0,Columbia,Los Rostros del Desempleo,Detalles Redacción Los Rostros del Desempleo D...,news,Faces of Unemployment,Editorial details Faces of Unemployment detail...
1,http://columbia.co.cr/index.php/reportajes/esp...,http://columbia.co.cr/,1.0,Columbia,Con los pies en la luna,Detalles Alejandro Arley Con los pies en la lu...,news,With the feet on the moon,Alejandro Arley Details Down to the moon detai...
2,http://columbia.co.cr/index.php/noticias/inter...,http://columbia.co.cr/,1.0,Columbia,El salario mínimo de Venezuela toca suelo: 2 d...,"Caracas, 30 ago (EFE).- Hoy, por primera vez, ...",news,Venezuela's minimum wage floor plays: $ 2,"Caracas, Aug 30 (EFE) .- Today, for the first ..."
3,http://columbia.co.cr/index.php/deportes/futbo...,http://columbia.co.cr/,1.0,Columbia,"Jafet Soto ""Traían una estrategia de meter un ...","El técnico del Club Sport Herediano, Jafet Sot...",news,"Jafet Soto ""They brought a strategy to score a...","The coach of Club Sport Herediano, Jafet Soto ..."
4,http://columbia.co.cr/index.php/deportes/futbo...,http://columbia.co.cr/,1.0,Columbia,El Waterhouse deja fuera al Herediano en los p...,El Club Sport Herediano y el Waterhouse defini...,news,Waterhouse leaves out the Herediano on penalties,The Club Sport Herediano and Waterhouse series...


##Lowercasing

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
df2 = df.apply(lambda x: x.astype(str).str.lower())
df2.head()

,URLNoticia,Fuente,Clasificacion,MedioPeriodistico,Titulo,Texto,TipoSitio,TituloIngles,TextoIngles
0,http://columbia.co.cr/index.php/reportajes/esp...,http://columbia.co.cr/,1.0,columbia,los rostros del desempleo,detalles redacción los rostros del desempleo d...,news,faces of unemployment,editorial details faces of unemployment detail...
1,http://columbia.co.cr/index.php/reportajes/esp...,http://columbia.co.cr/,1.0,columbia,con los pies en la luna,detalles alejandro arley con los pies en la lu...,news,with the feet on the moon,alejandro arley details down to the moon detai...
2,http://columbia.co.cr/index.php/noticias/inter...,http://columbia.co.cr/,1.0,columbia,el salario mínimo de venezuela toca suelo: 2 d...,"caracas, 30 ago (efe).- hoy, por primera vez, ...",news,venezuela's minimum wage floor plays: $ 2,"caracas, aug 30 (efe) .- today, for the first ..."
3,http://columbia.co.cr/index.php/deportes/futbo...,http://columbia.co.cr/,1.0,columbia,"jafet soto ""traían una estrategia de meter un ...","el técnico del club sport herediano, jafet sot...",news,"jafet soto ""they brought a strategy to score a...","the coach of club sport herediano, jafet soto ..."
4,http://columbia.co.cr/index.php/deportes/futbo...,http://columbia.co.cr/,1.0,columbia,el waterhouse deja fuera al herediano en los p...,el club sport herediano y el waterhouse defini...,news,waterhouse leaves out the herediano on penalties,the club sport herediano and waterhouse series...


##Eliminación de Ruido - información geoespacial de la noticia

In [ ]:
def eliminarTags(text):
  #remueve san josé, 29 ago (elmundo.cr) san francisco (ee.uu.), 2 ago (efe).- san josé, 29 ago (elmundo.cr)- 
  return re.sub(".*,? [0-9]{1,2} [a-z.]{3,4},?\ \(.*\)?.-?|\(afp\)-?|.*,? ?[a-z.]{3,4} [0-9]{1,2},? \(.*\) \ ?-?|redacción.|\n\n"," ",text)


In [ ]:
df2['Texto'] = df2.apply(lambda row: eliminarTags(row['Texto']), axis=1)
df2['TextoIngles'] = df2.apply(lambda row: eliminarTags(row['TextoIngles']), axis=1)
df2.head()

,URLNoticia,Fuente,Clasificacion,MedioPeriodistico,Titulo,Texto,TipoSitio,TituloIngles,TextoIngles
0,http://columbia.co.cr/index.php/reportajes/esp...,http://columbia.co.cr/,1.0,columbia,los rostros del desempleo,detalles los rostros del desempleo detalles c...,news,faces of unemployment,editorial details faces of unemployment detail...
1,http://columbia.co.cr/index.php/reportajes/esp...,http://columbia.co.cr/,1.0,columbia,con los pies en la luna,detalles alejandro arley con los pies en la lu...,news,with the feet on the moon,alejandro arley details down to the moon detai...
2,http://columbia.co.cr/index.php/noticias/inter...,http://columbia.co.cr/,1.0,columbia,el salario mínimo de venezuela toca suelo: 2 d...,con el dólar estadounidense alrededor de los...,news,venezuela's minimum wage floor plays: $ 2,".- today, for the first time, millions of ven..."
3,http://columbia.co.cr/index.php/deportes/futbo...,http://columbia.co.cr/,1.0,columbia,"jafet soto ""traían una estrategia de meter un ...","el técnico del club sport herediano, jafet sot...",news,"jafet soto ""they brought a strategy to score a...","the coach of club sport herediano, jafet soto ..."
4,http://columbia.co.cr/index.php/deportes/futbo...,http://columbia.co.cr/,1.0,columbia,el waterhouse deja fuera al herediano en los p...,el club sport herediano y el waterhouse defini...,news,waterhouse leaves out the herediano on penalties,the club sport herediano and waterhouse series...


##Lematización

In [ ]:
#pip install es-lemmatizer
!python -m spacy download es
!python -m spacy download en


✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/es_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/es
You can now load the model via spacy.load('es')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
from es_lemmatizer import lemmatize
import spacy

In [ ]:
nlp_espannol = spacy.load("es")
nlp_ingles = spacy.load("en")

In [ ]:
text1 = "Con estos fines, la Dirección de Gestión y Control Financiero monitorea la posición de capital del Banco y utiliza los mecanismos para hacer un eficiente manejo del capital."
def lematizar(texto,npl):
  lemas = []
  text = []
  for token in nlp(texto): 
    lemas.append(token.lemma_)
    text.append(token.text)
      #print(token.text, token.lemma_, token.pos_)
  return lemas

In [ ]:
#lematizar(text1,nlp_espannol)
#doc = nlp_espannol(text1)
#for att in dir(doc):
#    print (att, getattr(doc,att))

In [ ]:
def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

In [ ]:
df2['textoLematizadoIngles'] = df2.apply(lambda row: lematizar(row['TextoIngles'],nlp_ingles), axis=1)
df2['textoLematizado'] = df2.apply(lambda row: lematizar(row['Texto'],nlp_espannol), axis=1)

In [ ]:
df2.head()

,URLNoticia,Fuente,Clasificacion,MedioPeriodistico,Titulo,Texto,TipoSitio,TituloIngles,TextoIngles,textoLematizadoIngles,textoLematizado
0,http://columbia.co.cr/index.php/reportajes/esp...,http://columbia.co.cr/,1.0,columbia,los rostros del desempleo,detalles los rostros del desempleo detalles c...,news,faces of unemployment,editorial details faces of unemployment detail...,"[editorial, details, faz, of, unemployment, de...","[detalle, , lo, rostro, del, desempleo, detal..."
1,http://columbia.co.cr/index.php/reportajes/esp...,http://columbia.co.cr/,1.0,columbia,con los pies en la luna,detalles alejandro arley con los pies en la lu...,news,with the feet on the moon,alejandro arley details down to the moon detai...,"[alejandro, arley, details, down, to, the, moo...","[detalle, alejandro, arley, con, lo, pie, en, ..."
2,http://columbia.co.cr/index.php/noticias/inter...,http://columbia.co.cr/,1.0,columbia,el salario mínimo de venezuela toca suelo: 2 d...,con el dólar estadounidense alrededor de los...,news,venezuela's minimum wage floor plays: $ 2,".- today, for the first time, millions of ven...","[ , .-, today, ,, for, the, first, timar, ,, m...","[ , con, el, dólar, estadounidense, alrededor..."
3,http://columbia.co.cr/index.php/deportes/futbo...,http://columbia.co.cr/,1.0,columbia,"jafet soto ""traían una estrategia de meter un ...","el técnico del club sport herediano, jafet sot...",news,"jafet soto ""they brought a strategy to score a...","the coach of club sport herediano, jafet soto ...","[the, coach, of, club, sport, herediano, ,, ja...","[el, técnico, del, club, sport, herediano, ,, ..."
4,http://columbia.co.cr/index.php/deportes/futbo...,http://columbia.co.cr/,1.0,columbia,el waterhouse deja fuera al herediano en los p...,el club sport herediano y el waterhouse defini...,news,waterhouse leaves out the herediano on penalties,the club sport herediano and waterhouse series...,"[the, club, sport, herediano, and, waterhouse,...","[el, club, sport, herediano, y, el, waterhouse..."


##Tokenizar noticias

In [ ]:
#noticias_ingles = df2.TextoIngles.str.cat(sep=' ')
#function to split text into word
#tokens_ingles = word_tokenize(noticias_ingles)
#tokens_ingles_2=[token.lower() for token in tokens_ingles if token.isalpha()] # remueve los signos de puntuación y números
#vocabulario_ingles = set(tokens_ingles)
#print(len(vocabulario_ingles))

In [ ]:
#noticias_espannol = df2.Texto.str.cat(sep=' ')
#function to split text into word
#tokens_espannol = word_tokenize(noticias_espannol)
#vocabulario_espannol = set(tokens_espannol)
#print(len(vocabulario_espannol))

###Tokenización del dataset

In [ ]:
#Se tokeniza cel texto de cada noticia en español por separado y se almacena en una nueva variable
df2['textoTokenizado'] = df2.apply(lambda row: nltk.word_tokenize(row['Texto']), axis=1)
#Se tokeniza cel texto de cada noticia en ingles por separado y se almacena en una nueva variable
df2['textoTokenizadoIngles'] = df2.apply(lambda row: nltk.word_tokenize(row['TextoIngles']), axis=1)
df2.head()

,URLNoticia,Fuente,Clasificacion,MedioPeriodistico,Titulo,Texto,TipoSitio,TituloIngles,TextoIngles,textoLematizadoIngles,textoLematizado,textoTokenizado,textoTokenizadoIngles
0,http://columbia.co.cr/index.php/reportajes/esp...,http://columbia.co.cr/,1.0,columbia,los rostros del desempleo,detalles los rostros del desempleo detalles c...,news,faces of unemployment,editorial details faces of unemployment detail...,"[editorial, details, faz, of, unemployment, de...","[detalle, , lo, rostro, del, desempleo, detal...","[detalles, los, rostros, del, desempleo, detal...","[editorial, details, faces, of, unemployment, ..."
1,http://columbia.co.cr/index.php/reportajes/esp...,http://columbia.co.cr/,1.0,columbia,con los pies en la luna,detalles alejandro arley con los pies en la lu...,news,with the feet on the moon,alejandro arley details down to the moon detai...,"[alejandro, arley, details, down, to, the, moo...","[detalle, alejandro, arley, con, lo, pie, en, ...","[detalles, alejandro, arley, con, los, pies, e...","[alejandro, arley, details, down, to, the, moo..."
2,http://columbia.co.cr/index.php/noticias/inter...,http://columbia.co.cr/,1.0,columbia,el salario mínimo de venezuela toca suelo: 2 d...,con el dólar estadounidense alrededor de los...,news,venezuela's minimum wage floor plays: $ 2,".- today, for the first time, millions of ven...","[ , .-, today, ,, for, the, first, timar, ,, m...","[ , con, el, dólar, estadounidense, alrededor...","[con, el, dólar, estadounidense, alrededor, de...","[.-, today, ,, for, the, first, time, ,, milli..."
3,http://columbia.co.cr/index.php/deportes/futbo...,http://columbia.co.cr/,1.0,columbia,"jafet soto ""traían una estrategia de meter un ...","el técnico del club sport herediano, jafet sot...",news,"jafet soto ""they brought a strategy to score a...","the coach of club sport herediano, jafet soto ...","[the, coach, of, club, sport, herediano, ,, ja...","[el, técnico, del, club, sport, herediano, ,, ...","[el, técnico, del, club, sport, herediano, ,, ...","[the, coach, of, club, sport, herediano, ,, ja..."
4,http://columbia.co.cr/index.php/deportes/futbo...,http://columbia.co.cr/,1.0,columbia,el waterhouse deja fuera al herediano en los p...,el club sport herediano y el waterhouse defini...,news,waterhouse leaves out the herediano on penalties,the club sport herediano and waterhouse series...,"[the, club, sport, herediano, and, waterhouse,...","[el, club, sport, herediano, y, el, waterhouse...","[el, club, sport, herediano, y, el, waterhouse...","[the, club, sport, herediano, and, waterhouse,..."


In [ ]:
#frequency_dist_espannol = nltk.FreqDist(tokens_espannol)
#sorted(frequency_dist_espannol,key=frequency_dist_espannol.__getitem__, reverse=True)[0:50]

##Eliminación de Stop Words

In [ ]:
from nltk.corpus import stopwords

In [ ]:
#stop_words_ingles = set(stopwords.words('english'))
#stop_words_espannol = set(stopwords.words('spanish'))
#stop_words_ingles
#stop_words_espannol

In [ ]:
def removerStopWords(tokens, stopwords):
  return [w for w in tokens if not w in stopwords]  

Construir lista de stop words propios

In [ ]:
#df2['textoTokenizadoIngles'] = df2.apply(lambda row: removerStopWords(row['textoTokenizadoIngles'],stop_words_ingles), axis=1)
#df2['textoTokenizado'] = df2.apply(lambda row: removerStopWords(row['textoTokenizado'],stop_words_espannol), axis=1)
#df2.head()

In [ ]:
tokens_espannol_sin_stop_words = [w for w in tokens_espannol if not w in stop_words_espannol]
tokens_ingles_sin_stop_words = [w for w in tokens_ingles if not w in stop_words_ingles]
tokens_ingles_sin_stop_words_v2 = [w for w in tokens_ingles_2 if not w in stop_words_ingles]
tokens_ingles_sin_stop_words_v3 = removerStopWords(tokens_ingles_2,stop_words_ingles)

##Eliminación de ruido incluyendo caracteres  / Este paso no se ha aplicado


In [ ]:
def removerSignosPuntuacion(text):
    """Basic cleaning of texts."""
    
    # remove html markup
    text=re.sub("(<.*?>)","",text)
    
    #remove non-ascii and digits
    text=re.sub("(\\W|\\d)"," ",text)   

    #text=text.strip()
    return text

In [ ]:
def eliminarRuido(tokens):
  return [removerSignosPuntuacion(w) for w in tokens]

##Stemming o Derivación 
Cambiarlo por lematización

In [ ]:
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
porter_stemmer=PorterStemmer()
stemmer_espannol = SnowballStemmer('spanish')
stemmer_ingles = SnowballStemmer('english')


In [ ]:
def derivarTexto(words, stemmer):
  return [stemmer.stem(word=word) for word in words]

In [ ]:
#df2['textoTokenizadoInglesStem'] = df2.apply(lambda row: derivarTexto(row['textoTokenizadoIngles'],stemmer_ingles), axis=1)
#df2['textoTokenizadoStem'] = df2.apply(lambda row: derivarTexto(row['textoTokenizado'],stemmer_espannol), axis=1)
#df2.head()

In [ ]:
#frequency_dist_ingles = nltk.FreqDist(tokens_ingles_sin_stop_words_v2)
#sorted(frequency_dist_ingles,key=frequency_dist_ingles.__getitem__, reverse=True)[0:50]

In [ ]:
def removeSpecialChar(text):
  return re.sub("\[|\]|,|\'|\.|\$|\¢|\&|\#|\"|\x97|\x96|\x95|\x85|\x94|\x93|\:|\x92\|\ufeff1|\@", "",text)

In [ ]:
df2 = df2.astype(str)

In [ ]:
df2['textoPreprocesadoIngles'] = df2.apply(lambda row: removeSpecialChar(row['textoLematizadoIngles']), axis=1)
df2['textoPreprocesado'] = df2.apply(lambda row: removeSpecialChar(row['textoLematizado']), axis=1)
df2.head()

,URLNoticia,Fuente,Clasificacion,MedioPeriodistico,Titulo,Texto,TipoSitio,TituloIngles,TextoIngles,textoLematizadoIngles,textoLematizado,textoTokenizado,textoTokenizadoIngles,textoPreprocesadoIngles,textoPreprocesado
0,http://columbia.co.cr/index.php/reportajes/esp...,http://columbia.co.cr/,1.0,columbia,los rostros del desempleo,detalles los rostros del desempleo detalles c...,news,faces of unemployment,editorial details faces of unemployment detail...,"['editorial', 'details', 'faz', 'of', 'unemplo...","['detalle', ' ', 'lo', 'rostro', 'del', 'desem...","['detalles', 'los', 'rostros', 'del', 'desempl...","['editorial', 'details', 'faces', 'of', 'unemp...",editorial details faz of unemployment details ...,detalle lo rostro del desempleo detalle crea...
1,http://columbia.co.cr/index.php/reportajes/esp...,http://columbia.co.cr/,1.0,columbia,con los pies en la luna,detalles alejandro arley con los pies en la lu...,news,with the feet on the moon,alejandro arley details down to the moon detai...,"['alejandro', 'arley', 'details', 'down', 'to'...","['detalle', 'alejandro', 'arley', 'con', 'lo',...","['detalles', 'alejandro', 'arley', 'con', 'los...","['alejandro', 'arley', 'details', 'down', 'to'...",alejandro arley details down to the moon detai...,detalle alejandro arley con lo pie en lo luna ...
2,http://columbia.co.cr/index.php/noticias/inter...,http://columbia.co.cr/,1.0,columbia,el salario mínimo de venezuela toca suelo: 2 d...,con el dólar estadounidense alrededor de los...,news,venezuela's minimum wage floor plays: $ 2,".- today, for the first time, millions of ven...","[' ', '.-', 'today', ',', 'for', 'the', 'first...","[' ', 'con', 'el', 'dólar', 'estadounidense',...","['con', 'el', 'dólar', 'estadounidense', 'alre...","['.-', 'today', ',', 'for', 'the', 'first', 't...",- today for the first timar millions of ve...,con el dólar estadounidense alrededor de lo...
3,http://columbia.co.cr/index.php/deportes/futbo...,http://columbia.co.cr/,1.0,columbia,"jafet soto ""traían una estrategia de meter un ...","el técnico del club sport herediano, jafet sot...",news,"jafet soto ""they brought a strategy to score a...","the coach of club sport herediano, jafet soto ...","['the', 'coach', 'of', 'club', 'sport', 'hered...","['el', 'técnico', 'del', 'club', 'sport', 'her...","['el', 'técnico', 'del', 'club', 'sport', 'her...","['the', 'coach', 'of', 'club', 'sport', 'hered...",the coach of club sport herediano jafet soto ...,el técnico del club sport herediano jafet sot...
4,http://columbia.co.cr/index.php/deportes/futbo...,http://columbia.co.cr/,1.0,columbia,el waterhouse deja fuera al herediano en los p...,el club sport herediano y el waterhouse defini...,news,waterhouse leaves out the herediano on penalties,the club sport herediano and waterhouse series...,"['the', 'club', 'sport', 'herediano', 'and', '...","['el', 'club', 'sport', 'herediano', 'y', 'el'...","['el', 'club', 'sport', 'herediano', 'y', 'el'...","['the', 'club', 'sport', 'herediano', 'and', '...",the club sport herediano and waterhouse serie ...,el club sport herediano y el waterhouse defini...


#Normalizar texto

In [ ]:
#https://carlosvin.github.io/es/posts/normalize-text-py/
def normalize_text ( text ):
  return normalize('NFKD', text).encode('ASCII', 'ignore')

In [ ]:
#df2['textoPreprocesadoIngles'] = df2.apply(lambda row: normalize_text(row['textoPreprocesadoIngles']), axis=1)
#df2['textoPreprocesado'] = df2.apply(lambda row: normalize_text(row['textoPreprocesado']), axis=1)
#df2.head()

In [ ]:
#df2.to_csv(rootDir+'/ConjuntoDatosPreprocesados_20190922.csv')
df2.to_excel(rootDir+'/ConjuntoDatosPreprocesados_20190922_lematizado.xlsx')